In [455]:
import nltk 
import pandas as pd

import numpy as np
import csv
import openai
from bertopic.representation import OpenAI

import sqlite3

### data import

In [ ]:
# topic_df = pd.read_csv("C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\tgpt_feb_2_jul_topic_mapping_by_llama.csv")
# topic_df.shape

# df = pd.read_csv("C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\feb_2_aug_topic_db.csv", encoding = 'utf-8')

In [509]:
file_path = "C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\feb_2_aug_topic_db.csv"
with open (file_path, 'r', encoding='ISO-8859-1') as f:
    csv_file = csv.reader(f)
    header = next(csv_file)
    rows = [row for row in csv_file]
df = pd.DataFrame(rows, columns=header)

In [510]:
df.head()

,message_id,user_query_x,created_at,topic_y,month,session_id
0,27ad4be3-ab42-4fe9-945f-b4a700edf3b1,but the put open interest is higher than call ...,6/16/2024 14:07,Options Trading Strategies,6,926e5219-2760-4c63-85f3-a8d217115fc5
1,0cada57b-ef39-418d-9486-46d3135b6c1e,Which stocks will increase 2% on next two week...,6/16/2024 14:12,Options Trading Strategies,6,0fce8430-4a6f-4ad9-8c7e-35b9973ac490
2,4b1b1dac-04c4-4c69-a809-05cabef3529f,RapidAdvanceSolutions.com - What do I use for ...,7/3/2024 1:28,Non-Financial/Unrelated Topics,7,3e3ad757-8cc9-4bc7-b3d3-7173e6f41901
3,4616cd71-3846-4b3d-9d38-54e9bc7655b3,Yes,6/16/2024 14:12,Non-Financial/Unrelated Topics,6,0fce8430-4a6f-4ad9-8c7e-35b9973ac490
4,07dbbadf-37fd-4cf2-9d99-4d5d73d872d5,Im on a demo account,6/16/2024 15:20,Trading Terminology & Tools,6,52dc0c3a-4f46-418b-84e5-b4b36d6921f1


In [513]:
df.columns

Index(['message_id', 'user_query_x', 'created_at', 'topic_y', 'month',
       'session_id'],
      dtype='object')

In [533]:
df1 = df.loc[df['month'].isin(['8','9']), ['session_id', 'user_query_x', 'created_at']]
df1.shape

(105839, 3)

In [535]:
df1_sorted = df1.sort_values(by = ['session_id', 'created_at'], ascending=[True, True])

In [537]:
df1_sorted.head(50)

,session_id,user_query_x,created_at
339355,00010369-2ee4-459b-9080-43c6793c32ff,What's the least expensive crypto coin to trad...,8/3/2024 2:01
339356,00010369-2ee4-459b-9080-43c6793c32ff,"The crypto market which one cost the less in ""...",8/3/2024 2:02
339358,00010369-2ee4-459b-9080-43c6793c32ff,"Thank you, you're always a big help",8/3/2024 2:03
394108,0003c604-45a5-4f1d-8b87-b684ef3641e4,Ciao,8/22/2024 17:08
356592,00054a16-b0cb-4c2e-84c1-505a7e859899,Get me trades for today to buy for weekend gai...,8/9/2024 19:25
356600,00054a16-b0cb-4c2e-84c1-505a7e859899,"Which of these is highest probability, high bu...",8/9/2024 19:30
356611,00054a16-b0cb-4c2e-84c1-505a7e859899,Give me your best picks for trades on blnd and...,8/9/2024 19:35
356624,00054a16-b0cb-4c2e-84c1-505a7e859899,If they go up what is the profit possible on e...,8/9/2024 19:38
356630,00054a16-b0cb-4c2e-84c1-505a7e859899,Possible profit from option on blnd and fivn a...,8/9/2024 19:41
424434,0006538f-33de-4fb9-8f57-19835fb2e110,Top 3 Call option contracts in the AI hardware...,9/1/2024 4:59


In [539]:
session_time = df1_sorted.groupby('session_id')['created_at'].apply(min).reset_index(name='session_time')

In [541]:
df_session = df1_sorted.groupby('session_id')['user_query_x'].apply(list).reset_index(name='quer_list')
df_session['session_time'] = session_time['session_time']
df_session.shape

(35267, 3)

In [547]:
df_session.head(10)

,session_id,quer_list,session_time
0,00010369-2ee4-459b-9080-43c6793c32ff,[What's the least expensive crypto coin to tra...,8/3/2024 2:01
1,0003c604-45a5-4f1d-8b87-b684ef3641e4,[Ciao],8/22/2024 17:08
2,00054a16-b0cb-4c2e-84c1-505a7e859899,[Get me trades for today to buy for weekend ga...,8/9/2024 19:25
3,0006538f-33de-4fb9-8f57-19835fb2e110,[Top 3 Call option contracts in the AI hardwar...,9/1/2024 4:59
4,0007bc3e-d9ec-4a4f-b79c-b2f7069d0a4e,"[What stock made the most money today, What pe...",8/23/2024 20:45
5,000cf849-868c-444a-adf9-1bc84655da4b,[Weekly Swing Trade: Find me an options trade ...,8/14/2024 16:44
6,000e11e4-592b-414a-8825-2a217a663853,[What is the current market sentiment for ARM....,8/6/2024 4:49
7,000e3867-a447-47da-81fa-91a9c22b3675,[Whats my stop loss and take profit for LCID ...,8/6/2024 18:29
8,0011fb82-cf49-4349-b7cf-760b191d0f4d,[Does the healthcare stock THC have strong fun...,8/16/2024 12:38
9,00127f9d-ef7c-4124-a35b-c98f30c2eca2,"[Is royal Caribbean a good buy?, Entry point f...",8/28/2024 19:02


In [549]:
docs = df_session['quer_list'].tolist()[:100000]
len(docs)

35267

### LLM sentiment analysis

In [551]:
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
MAX_CONCURRENCY = 32

In [553]:
semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
server_host = "http://100.84.186.114:8080"
# model = "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4"
model="gpt-3.5-turbo"
# api_key = "TSGS@LLM"
api_key="sk-proj-DlLitL7d2xVm..."
async_client = AsyncOpenAI(
    #base_url=server_host + "/v1",
    api_key=api_key,
)

In [ ]:
# objective_prompt = """
#         Objective:
#         You are a professional stock market analyst.
#         Given user input, your task is to analyze sentiment and generate sentiment score for target company.
#         Additionally, please provide a short, concise and clear reasoning for entire anlysis.
#         """

#         steps_prompt = f"""
#         Instructions:
#         1. Read headline and preview thoroughly to understand full context.
#         2. Identify the target company being analyzed based on given ticker name {ticker}. Ensure you concentrate on the target company's portrayal without being influenced by other companies' sentiment.
#         3. Assign an overall sentiment for {ticker}. Please choose an answer from 'Positive, Neutral, Negative'. 
#         4. Assign a sentiment score from -100 to 100 for {ticker}.
#             * -100 to -1 represents Negative sentiment (lower score indicates stronger negativity)
#             * 0 represents Neutral sentiment
#             * 1 to 100 represents Positive sentiment (higher score indicates stronger positivity)
#         5. Reflect on your analysis and make changes if necessary.
#         6. Generate final output following below output format.
#         """

#         output_prompt = f"""
#         Output Format:
#         [Sentiment]: Positive, Neutral, or Negative
#         [Score]: ...
#         [Reasoning]: ...
#         """

In [ ]:
# prompt = f"""
# You are an NLP expert specializing in sentiment analysis. 
# Your task involves analyzing user queries, which may be either new messages or follow-up messages.
# Your job is:
# step 1: to classify if each query is "new message" or "followup message";
# step 2: for followup message, perform sentiment analysis and assign 'positive', 'negative' or 'neutral', following this definition:
#     * 'positive': user seems satisfied with the answer given, by saying 'yes','thank you','go ahead', 'implemend it', or the tone sounds positive;
#     * 'negative': user seems unsatisfied with the answer given, by saying 'no' or the tone sounds negative;
#     * 'neutral': user seems neither obviously happy or unhappy, or you cannot tell from the  user query
    
# Please generate output in one word: positive, neutral, or negative
# followed by a reasoning analysis of less than 10 words. 

# """

# # [message type]: new or followup
# # [sentiment]: positive, neutral, or negative
# prompt += "You are given following user query: {message}"

In [ ]:
# prompt = f"""
# You are an NLP expert specializing in sentiment analysis. 
# Your task involves analyzing user queries, which may be either new messages or follow-up messages.

# Definitions:
# New Message: A query that does not reference previous interactions or context. For example:
#     "What are the current trends in the stock market?"
#     "Can you explain how options trading works?"
# Follow-up Message: A query that references or builds upon previous interactions. For example:
#     "Can you provide more details on that strategy?"
#     "I liked your last analysis; can we proceed with it?"

# Tasks:
# 1. Classify the Query:
#     Determine if the query is a "new message" or a "follow-up message."
# 2. Sentiment Analysis for Follow-up Messages: If the query is a follow-up message, perform sentiment analysis and classify it as 'positive', 'negative', or 'neutral' based on the following criteria:
#     Positive: The user expresses satisfaction. Examples include:
#         "Thank you for that insightful analysis!"
#         "Yes, let's implement this strategy!"
#     Negative: The user expresses dissatisfaction. Examples include:
#         "No, that advice didn’t work."
#         "I’m not happy with those results."
#     Neutral: The user is neither clearly satisfied nor dissatisfied, or the sentiment is unclear. Examples include:
#         "Can you clarify that recommendation?"
#         "I’m not sure about the next steps."
# Output:
#     Provide the sentiment classification in one word: positive, neutral, or negative.
#     Include a brief reasoning analysis of less than 10 words.
    
# Example Queries:
#     Query: "What are the current trends in the stock market?"
#     Output: New message
#     Query: "Can you provide more details on that strategy?"
#     Output: Neutral
#     Query: "Thank you for your market insights!"
#     Output: Positive - User expresses satisfaction.
#     Query: "No, that advice didn’t work."
#     Output: Negative - User expresses dissatisfaction.
#     Query: "I’m not sure about the next steps."
#     Output: Neutral - Unclear sentiment.
# """

# prompt += "You are given following user query: {message}"

In [ ]:
prompt = f"""
You are an NLP expert specializing in sentiment analysis based on a session of user queries. 
Each session may consists of multiple user queries, each query may be a question or a response to an answer provided by a machine, but only the user's text is provided. 
Focus on identifying the sentiment conveyed in each session, whether it is positive, negative, or neutral.

Please base on the following criteria:
    Positive: The user expresses satisfaction. Examples include:
        "Thank you for that insightful analysis!"
        "Yes, let's implement this strategy!"
    Negative: The user expresses dissatisfaction. Examples include:
        "No, that advice didn’t work."
        "I’m not happy with those results."
    Neutral: The user is neither clearly satisfied nor dissatisfied, or the sentiment is unclear. Examples include:
        "What are the current trends in the stock market?"
        "Can you clarify that recommendation?"
        "I’m not sure about the next steps."
Output:
    Strictly limit your sentiment classification answer in one word: positive, neutral, or negative.

    
Example Queries:
    Query: "What are the current trends in the stock market?"
    Output: Neutral
    Query: "Can you provide more details on that strategy?"
    Output: Neutral
    Query: "Thank you for your market insights!"
    Output: Positive
    Query: "No, that advice didn’t work."
    Output: Negative
    Query: "I’m not sure about the next steps."
    Output: Neutral
"""
#    Include a brief reasoning analysis of less than 10 words.
    # Query: "What are the current trends in the stock market?"
    # Output: Neutral - Just a question.
    # Query: "Can you provide more details on that strategy?"
    # Output: Neutral - Just a question.
    # Query: "Thank you for your market insights!"
    # Output: Positive - User expresses satisfaction.
    # Query: "No, that advice didn’t work."
    # Output: Negative - User expresses dissatisfaction.
    # Query: "I’m not sure about the next steps."
    # Output: Neutral - Unclear sentiment.
prompt += "You are given- Query: {message} Output:"

In [555]:
prompt = f"""
You are an NLP expert specializing in sentiment analysis based on a session of user queries. 
Each session may consists of multiple user queries, each query may be a question or a response to an answer provided by a machine, but only the user's text is provided. 
Focus on identifying the sentiment conveyed in each session, whether it is positive, negative, or neutral.
The sentiment should be purely based on reaction to the quality or relevance of the machine's answers, and ignore any sentiment related to external factors such as market conditions, portfolios, or trading.

Please base on the following criteria:
    Positive: The user expresses satisfaction in at least one of the query. Examples include:
        "Thank you for that insightful analysis!"
        "Yes, let's implement this strategy!"
    Negative: The user expresses dissatisfaction in at least one of the query. Examples include:
        "No, that advice didn’t work."
        "I’m not happy with those results."
    Neutral: The user is neither satisfied nor dissatisfied, or the sentiment is unclear. Examples include:
        "What are the current trends in the stock market?"
        "Can you clarify that recommendation?"
        "I’m not sure about the next steps."
        
**Output Requirements:**
1. **Sentiment:** Provide a **one-word** sentiment classification: **Positive**, **Neutral**, or **Negative**.
2. **Reasoning:** Provide a concise and insightful explanation for the sentiment. If the sentiment is positive, summarize what user is happy with; if negative,  what user is not happy about and how do you suggest to improve 

**Formatting:**
- Start with "Sentiment:" followed by the classification.
- Continue with "Reasoning:" followed by the explanation.

**Examples for Guidance:**
**Session 1:**
    Query 1: "What are the current trends in the stock market?"
    Query 2: "Thank you for the detailed overview."
    Query 3: "whats the outlook for PLUG?"
**Output:**
  Sentiment: Positive
  Reasoning: user expresses satisfaction with the detailed overview after asking about stock market trend

**Session 2:**
    Query 1: "Give me the best stocks to trade."
    Query 2: "No, that is not about right."
**Output:**
  Sentiment: Negative
  Reasoning: user expresses dissatisfaction with the advice given, by saying the recommended stocks to trade are not right. 
"""

prompt += "You are given- **session**: {message} Overall Sentiment:"

In [195]:
# message = ['How many shares of Tsla was sold',
#  'How many shares of TSLA was sold yesterday',
#  'How many of those share were a sell order ']
# print(prompt.format(message = message))

In [557]:
async def process_review(text):
    async with semaphore:
        try:
            input = [{"role": "user", "content": prompt.format(message=text)}]
            response = await async_client.chat.completions.create(
                model=model,
                messages=input,
                temperature=0,
                logprobs = True
            )
            res = response.choices[0].message.content
            #logprobs = response.choices[0].logprobs.content[0].logprob
            return res #, logprobs
        
        except Exception as e:
            print(f"Error: {e}. Failed {text}")
            return ""

In [559]:
async def main(docs):
    tasks = []
    for i, text in enumerate(docs):  # Use enumerate to get both index and text
        tasks.append((i, process_review(text)))  # Store index and task together

    # Use list comprehension to gather results and maintain index association
    res = await tqdm.gather(*(task[1] for task in tasks))

    # print('res:', res)
    # print(range(len(res)))
    # print(range(len(tasks)))
    length = len(docs)

    final_output = [(tasks[i][0], res[i]) for i in range(length)]
    return final_output

In [305]:
# message = [
#     ['How many shares of Tsla was sold, bcoz the market is really really positive','yes, thank you for that']
#     ,['Why is the stock market is down today ', 'when is inflation rate is coming out on this month and what can be effect for us stock market']
#     ,['What is the outlook for PLUG', 'Thank you', 'What is the outlook for WRAP?', 'what is the long term outlook for WRAP']
# ,['Fast turnaround stock market investment', 'Short-term trading', 'Yes please', 'Any other suggestions?', 'I would like more detailed analysis please', 'Thanks bud']
#     ,['Please provide me with the top 3 stocks that had inside days on Friday, July 19, 2024', "That isn't true none of these stocks were inside on the day for Friday July 19,2024", 'Please provide me with the top 3 stocks that had inside days on Friday, July 19, 2024', 'What are the top 3 stocks to buy for trade options on Monday July 22,2024', 'What are the top 3 small cap stocks to buy for trade options on Monday July 22,2024']       
#     ,['What stocks are best to swing trade today', 'Yes', 'What target price should I buy surmidics', 'What target price should I buy ainos', 'How to turn 100 into 10,000 today', 'What is the most stock is most profitable to day trade right now', 'Show me a daytrading strategy for these stocks']
# ,["Give me the past 5 quarters' EPS for TSLA.", 'what the best stock to but for profit ', 'what is the price prediction for terra luna ', 'what is the price prediction for terra luna classic ', 'what is the price prediction for bcel atreca ', 'what will happen to bbbyq shorts', 'will bbbyq share holders be compensated', 'what is the price prediction for amanx']
# ,['Do a comparison between NVDA and AMD.', 'fastest momentm stock with minimum risk today', 'fastest momentum stock low risk today May 15, 2024', 'User\nfastest momentum stock low risk today  with large volume May 15, 2024']
# ]
# await main(message)

In [561]:
output = await main(docs)

  1%|          | 376/35267 [00:19<25:09, 23.12it/s] 

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 79874 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Failed ['Clean the following code, optimize it, and add try exception  to it\n\nimport logging\nimport os\n\nimport pandas as pd\nimport talib\nimport tensorflow.keras as keras\n\n# Formats Position\nformat_position = lambda price: (\'-$\' if price < 0 else \'+$\') + \'{0:.2f}\'.format(abs(price))\n\n# Formats Currency\nformat_currency = lambda price: \'${0:.2f}\'.format(abs(price))\n\n\ndef show_train_result(result, val_position, initial_offset, strategy):\n    """ Displays training results\n    """\n    if val_position == initial_offset or val_position == 0.0:\n        logging.info(\'{}: Episode {}/{} - Train Position: {}  Val Position: USELESS  Train Loss: {:.4f}\'\n                     .format(strategy

 17%|█▋        | 5873/35267 [04:20<22:33, 21.71it/s]

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 127942 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Failed ['The rewards/Delta is not correctly calculated for the following code:\n\nimport gc\nimport logging\nimport numpy as np\nimport psutil\nfrom tqdm import tqdm\nfrom .ops import get_state\nfrom .utils import format_currency, format_position\n\ndef monitor_memory(threshold=80):\n    """Monitor memory usage and log if it exceeds the threshold."""\n    memory = psutil.virtual_memory()\n    if memory.percent > threshold:\n        logging.warning(f"Memory usage is at {memory.percent}% which is above {threshold}%.")\n        return True\n    return False\n\ndef execute_trade(action, agent, data, t, adjcp_field, trans_cost):\n    """Execute trade based on the action and return the profit and delta."""\n   

 50%|████▉     | 17520/35267 [13:29<11:21, 26.05it/s]  

...

 76%|███████▌  | 26827/35267 [20:20<05:35, 25.15it/s]

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 26673 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Failed ['What public companies produce top Alzheimer and prostate cancer ', 'which public company in the US produces isomex 30 mg', 'Barron is talking about Lumen ticker LUMN as a AI rocket that might explode. Elaborate on what they mean by that and analyze LUMN.', 'in the below transcript read between the lines to identify pros and cons of owning Lumens stock. Also determine the best price to acquire LUMN share and find out where it could be trading if the company acquires more contracts.', "In the below transcript read between lines to identify pros and cons of owning lumens stock. Also determine the best price to acquire LUMN share and find out where it could be trading if the company acquires additiona

 99%|█████████▉| 34951/35267 [26:22<00:14, 22.03it/s]

Error: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 40455 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Failed ['PUEDES AGREGAR A ESTE INDICADOR  UN  VIX OSCILADOR SMOOTH /@version=5 \nindicator("VIX Oscillator Smooth", shorttitle="VIX Osc Sm", overlay=false) \n\n// User Inputs \ntimeframe = input.timeframe("", title="Period Timeframe")  \nAvgLength = input.int(14, title="Average Length") \nSmoothLength = input.int(3, title="Smooth Length") \n\n// Securities \nticker = ticker.new(syminfo.prefix, syminfo.ticker, session.regular) \nvix = request.security("CBOE:VIX", timeframe, close)  \ncl = request.security(ticker, timeframe, close, lookahead=barmerge.lookahead_on) \n\n// Z-Score Calculation \n\n// Z Score VIX  \nvixhx = ta.sma(vix, AvgLength)  \nvixsd = ta.stdev(vix, AvgLength)  \nz_raw = (vix - vixhx) / vix

100%|██████████| 35267/35267 [26:36<00:00, 22.09it/s]


In [563]:
len(output)

35267

#### - post data processing

In [275]:
test = pd.DataFrame(output, columns = ['id','sentiment'])
test.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\sentiment_analysis\\100_test.csv")

In [565]:
import re

def map_to_sentiment(word):
    # Lowercase the word for easier comparison
    word = str(word).lower()
    
    # Regular expressions for matching similar words
    positive_patterns = r'\b(pos|posit|positive)\b'
    negative_patterns = r'\b(neg|negat|negative)\b'
    neutral_patterns = r'\b(neu|neutral)\b'

    # Check for matches and map accordingly
    if re.search(positive_patterns, word):
        return 'Positive'
    elif re.search(negative_patterns, word):
        return 'Negative'
    elif re.search(neutral_patterns, word):
        return 'Neutral'
    else:
        return 'Neutral'  # If no match, return None or handle accordingly

words = ['Positive.', 'negativity', 'Neutralize', 'unrelated', 'POSIT', 'NEG', 'neu', 'POSITIVE']
mapped_words = [map_to_sentiment(word) for word in words]
print(mapped_words)

['Positive', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Negative', 'Neutral', 'Positive']


In [567]:
df_output = pd.DataFrame(output, columns = ['id','sentiment'])
df_output.shape

(35267, 2)

In [569]:
df_output.head()

,id,sentiment
0,0,Sentiment: Positive\nReasoning: The user expre...
1,1,Sentiment: Neutral\nReasoning: The user's quer...
2,2,Sentiment: Neutral\nReasoning: The user is see...
3,3,Sentiment: Neutral\nReasoning: The user's quer...
4,4,Sentiment: Neutral\nReasoning: The user is inq...


In [571]:
df_output[['Sentiment', 'Reasoning']] = df_output['sentiment'].str.extract(r'Sentiment:\s*([^\n]+)\nReasoning:\s*(.*)')
df_output.head()

,id,sentiment,Sentiment,Reasoning
0,0,Sentiment: Positive\nReasoning: The user expre...,Positive,The user expresses satisfaction and gratitude ...
1,1,Sentiment: Neutral\nReasoning: The user's quer...,Neutral,"The user's query ""Ciao"" does not provide enoug..."
2,2,Sentiment: Neutral\nReasoning: The user is see...,Neutral,The user is seeking specific trading advice an...
3,3,Sentiment: Neutral\nReasoning: The user's quer...,Neutral,The user's query does not convey a clear senti...
4,4,Sentiment: Neutral\nReasoning: The user is inq...,Neutral,The user is inquiring about specific stocks an...


In [575]:
df_output['Sentiment'].value_counts()

Sentiment
Neutral     32047
Negative     1716
Positive     1231
Name: count, dtype: int64

In [577]:
df_map = df_output['Sentiment'].apply(map_to_sentiment).reset_index(name='map_to_sentiment')
df_map['Reasoning'] = df_output['Reasoning']
df_map.shape

(35267, 3)

In [579]:
df_session[['sentiment', 'reasoning']] = df_map[['map_to_sentiment', 'Reasoning']]
df_session.head()

,session_id,quer_list,session_time,sentiment,reasoning
0,00010369-2ee4-459b-9080-43c6793c32ff,[What's the least expensive crypto coin to tra...,8/3/2024 2:01,Positive,The user expresses satisfaction and gratitude ...
1,0003c604-45a5-4f1d-8b87-b684ef3641e4,[Ciao],8/22/2024 17:08,Neutral,"The user's query ""Ciao"" does not provide enoug..."
2,00054a16-b0cb-4c2e-84c1-505a7e859899,[Get me trades for today to buy for weekend ga...,8/9/2024 19:25,Neutral,The user is seeking specific trading advice an...
3,0006538f-33de-4fb9-8f57-19835fb2e110,[Top 3 Call option contracts in the AI hardwar...,9/1/2024 4:59,Neutral,The user's query does not convey a clear senti...
4,0007bc3e-d9ec-4a4f-b79c-b2f7069d0a4e,"[What stock made the most money today, What pe...",8/23/2024 20:45,Neutral,The user is inquiring about specific stocks an...


In [581]:
df_session.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\sentiment_analysis\\aug_session_sentiment0830.csv")

In [373]:
# add topic 

In [583]:
df_topic_list = df.groupby(['session_id'])['topic_y'].apply(list).reset_index(name = 'topic_list')
df_topic_list_explode = df_topic_list.explode('topic_list')
df_topic_list_explode = df_topic_list_explode[['session_id', 'topic_list']].drop_duplicates()
df_topic_list_explode.head()

,session_id,topic_list
0,00005a40-17f6-4f75-af23-91464e5dedb7,Stock Market Analysis
1,0000c413-41cf-4181-af22-6ff72b5d74e7,Options Trading Strategies
1,0000c413-41cf-4181-af22-6ff72b5d74e7,Market Sentiment & Trends
1,0000c413-41cf-4181-af22-6ff72b5d74e7,Non-Financial/Unrelated Topics
1,0000c413-41cf-4181-af22-6ff72b5d74e7,Stock Market Analysis


In [585]:
df_topic_list_explode.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\sentiment_analysis\\aug_session_topics0830.csv")

In [171]:
merge_df = pd.merge(df_session, df_session_sentiment, left_on = 'session_id', right_on = 'session_id', how = 'left')
merge_df.shape

(88431, 4)

In [173]:
merge_df.head()

,session_id,quer_list,sentiment,topic_list
0,00005a40-17f6-4f75-af23-91464e5dedb7,"[What should I invest on to male $1,000,000 of...",Positive,[Stock Market Analysis]
1,0000c413-41cf-4181-af22-6ff72b5d74e7,[Top 3 Call option contracts related to EV wit...,Neutral,"[Options Trading Strategies, Market Sentiment ..."
2,0001bf62-dc63-40ef-b940-7e3419384860,"[analyze hnd, analyze hnd.to, DO you have data...",Neutral,"[Technical Indicators & Analysis, Non-Financia..."
3,00027d51-b756-4dfd-8070-50ea23c93bdc,[Find me an options trade to make right now fo...,Negative,"[Options Trading Strategies, Options Trading S..."
4,00032951-f74d-4f67-ac8f-3655f9f2d153,"[What are you, Explain the Greeks as if you wo...",Positive,"[Non-Financial/Unrelated Topics, Options Tradi..."


In [179]:
merge_df.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\sentiment_analysis\\fen_jul_session_sentiment_with_topic.csv")

### tool extraction

In [591]:
df_tool = pd.read_csv("C:\\Users\\Liu Shi Peng\\Documents\\chat_messages_202409161505.csv")

In [593]:
df_tool.head()

,message_id,session_id,tools_used
0,85f5186b-4a1a-414e-8f76-2abde35f54f0,5454e817-c32d-4c1a-a9ef-0fc0d41fa02e,"[{""get_option_strategy_recommendation"": {""inpu..."
1,7e6a5c6f-ec03-4b32-aad8-67d99363760e,ef0763bf-4059-41c3-b974-ed951dfd423c,"[{""get_asset_price_info"": {""input"": {""ticker"":..."
2,3194fade-8a75-4abc-b0c6-be20c4db52e1,361858ce-1d9f-45fa-8a6b-a79035a4d555,"[{""get_stock_related_news"": {""input"": {""ticker..."
3,e05a0e53-246c-4e2c-bc5f-d6c107bbe99a,5f0de6ae-1741-4d88-894e-37032c34ae92,"[{""get_option_strategy_recommendation"": {""inpu..."
4,ead8e8f8-3308-49f6-8c61-a4b8f3942c95,1b3c4e72-c13c-47f8-8a9e-0452f62cacd9,"[{""get_asset_price_info"": {""input"": {""ticker"":..."


In [611]:
len(df_tool['session_id'].unique().tolist())

46630

In [503]:
# df2 = df[['session_id', 'message_id', 'tools_used']]
# df2.head()

In [597]:
import json
import ast

# def extract_tools(json_array_str):
#     json_array = json.loads(json_array_str)
#     return [list(item.keys())[0] for item in json_array]


def extract_tools(tools_str):
    try:
        # Use ast.literal_eval to safely evaluate the string representation of the list
        tools_list = ast.literal_eval(tools_str)
        return [list(item.keys())[0] for item in tools_list]
    except (ValueError, SyntaxError):
        # Return an empty list or None if parsing fails
        return []

In [599]:
df_tool['tools'] = df_tool['tools_used'].apply(extract_tools)
df2_tools = df_tool.explode('tools')
df2_tools = df2_tools[['session_id', 'tools']].drop_duplicates()
df2_tools.shape

(120759, 2)

In [601]:
df2_tools.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\sentiment_analysis\\aug_session_tools0830.csv")